In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from time import sleep
import pandas as pd
import re

# 启动浏览器
browser = webdriver.Edge()

# 访问二手房首页
browser.get('https://esf.fang.com/')
sleep(2)

# 鼠标悬停选择城市
target = browser.find_element(By.CSS_SELECTOR, "div.s4Box")
ActionChains(browser).move_to_element(target).perform()
sleep(1)

# 选择“北京”
browser.find_element(By.XPATH, "//div[@id='cityi010']/a[text()='北京']").click()
sleep(2)

# 依次选择“通州” -> “马驹桥”
for area in ["通州", "马驹桥"]:
    browser.find_element(By.LINK_TEXT, area).click()
    sleep(2)

# 存储房源数据
data_list = []

while True:
    try:
        container = browser.find_element(By.CSS_SELECTOR, "div.shop_list.shop_list_4")
        listings = container.find_elements(By.TAG_NAME, "dl")
    except NoSuchElementException:
        print("未找到房源信息")
        break

    for listing in listings:
        try:
            title = listing.find_element(By.XPATH, "./dd/h4/a/span").text.strip()
            details = listing.find_element(By.XPATH, "./dd/p[@class='tel_shop']").text.strip()
            total_price = listing.find_element(By.XPATH, "./dd[@class='price_right']/span[1]").text.strip()
            unit_price = listing.find_element(By.XPATH, "./dd[@class='price_right']/span[2]").text.strip()
            data_list.append({"标题": title, "户型_面积等": details, "总价": total_price, "单价": unit_price})
        except NoSuchElementException:
            continue

    try:
        next_btn = browser.find_element(By.LINK_TEXT, "下一页")
        next_btn.click()
        sleep(3)
    except NoSuchElementException:
        print("爬取完成")
        break

# 处理数据
df = pd.DataFrame(data_list)


def extract_area(info):
    match = re.search(r"\|\s*([\d.]+)㎡\s*\|", info)
    return float(match.group(1)) if match else None


df["平方数（㎡）"] = df["户型_面积等"].apply(extract_area)
df["总价（万）"] = df["总价"].str.replace("万", "", regex=False).astype(float)
df["单价（元/㎡）"] = df["单价"].str.replace("元/㎡", "", regex=False).astype(float)
df = df[["平方数（㎡）", "总价（万）", "单价（元/㎡）"]]

# 存储数据
df.to_csv("majuqiao_selling.csv", index=False)
print("数据保存完毕")
